In [ ]:
import numpy as np
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import json
import os


# Gets the moving average (mean) values 
def moving_avg(a, window):
    sum = 0.0
    result = list(0 for element in a)

    for i in range(0, window):
        sum = sum + a[i]
        result[i] = sum / (i+1)

    for i in range(window, len(a)):
        sum = sum - a[i-window] + a[i]
        result[i] = sum / window

    return result


# Gets all the time thresholds where there is a performance degradation
def get_relative_degradation(mv_avg_list, baseline_avg, x_values):
    relative_degradation = []
    temp = []
    for i, value in enumerate(mv_avg_list):
        if value > baseline_avg:
            temp.append(x_values[i])
        elif len(temp) > 0:
            relative_degradation.append(temp)
            temp = []
    if len(temp) > 0:
        relative_degradation.append(temp)
    return relative_degradation

for f in os.listdir('./'):
    task_id = ''
    if f.endswith('.json'):
        task_id = f.split('.')[0]
    else:
        continue
    # Import Rally task results, 
    # The files are in the same directory as this Jupyter notebook
    json_string = open('./{}.json'.format(task_id)).read()
    try:
        data = json.loads(json_string)
    except:
        continue

    for data_index, iteration in enumerate(data):

        try:
            # Get High level data
            iteration0_dur = data[data_index]["full_duration"]
            iteration0_load_dur = data[data_index]["load_duration"]
            hook_data = data[data_index]["hooks"][0]
            failure_injected = np.float64(
                hook_data["results"][0]["started_at"])\
                if len(hook_data["results"]) > 0 else None
            failure_description = hook_data["config"]["args"]["action"]
            scenario = data[data_index]["key"]["name"]

            # try to get baseline samples if a failure was indeed injected
            baseline_samples = [d for d in data[data_index]["result"] 
                                if d["timestamp"] < failure_injected 
                                and failure_injected is not None]

            # All Samples before and after failure injection
            all_samples = [d for d in data[data_index]["result"] 
                           if d["timestamp"]]

            # Max Operation Duration in iteration
            maxdur = max([d["duration"] for d in data[data_index]["result"]])

            # Success Operations duration
            success_dur = [d["duration"] for d in all_samples 
                           if len(d["error"]) == 0]
            # Failure Operation Duration
            failure_dur = [d["duration"] for d in all_samples 
                           if len(d["error"]) > 0]

            # Success Operations Timestamp
            success_timestamps = sorted([d["timestamp"] for d in all_samples 
                                         if len(d["error"]) == 0])
            # Failure Ops Timestamp
            failure_timestamps = sorted([d["timestamp"] for d in all_samples 
                                         if len(d["error"]) > 0])

            # All Durations
            all_dur = success_dur + failure_dur
            # All Timestamps Sorted ASC
            all_ts = sorted(success_timestamps + failure_timestamps)

            # Base timestamp tuple (start, end)
            base_ts = (all_ts[0],all_ts[-1])

            # Success Operation timestamps converted to seconds
            s_seconds = [ts - base_ts[0] for ts in success_timestamps]
            # Failure Operation timestamps converted to seconds
            f_seconds = [ts - base_ts[0] for ts in failure_timestamps]

            # All Operations Seconds (our X axis)
            all_seconds = sorted(s_seconds + f_seconds)

            baseline_dur = [d["duration"] for d in baseline_samples]
            baseline_ts = [d["timestamp"] for d in baseline_samples] 
            baseline_s = [ts - base_ts[0] for ts in baseline_ts]
            baseline_avg = np.mean(baseline_dur)

            # Calulate moving AVG (mean) in 10s time window 
            
            mv_avg = moving_avg(all_dur, 10) 
            
            percentile = np.percentile(baseline_dur, 95)
            
            # try to get mean duration from baseline if any
            mean_dur = [percentile for i in all_ts]
#             if  math.isnan(baseline_avg) and len(baseline_samples) < 80:
#                 mean_dur = [np.mean(all_dur) for i in all_ts] 

            # Get Relative degradation results
            rel_degradation = get_relative_degradation(mv_avg,
                                                       mean_dur[0],
                                                       all_seconds) 

            # PLOT the Results! #

            # plot success Ops
            success_ops, = plt.plot(s_seconds, 
                                    success_dur, 
                                    'b.',
                                    label="Success Ops")
            # plot fail ops
            fail_ops, = plt.plot(f_seconds,
                                 failure_dur,
                                 'r.',
                                 label="Fail Ops")

            # Plot Downtime area
            plt.fill_between(f_seconds,
                             0,
                             maxdur,
                             facecolor="red", 
                             color="red",
                             alpha=0.3,
                             label="First and Last Fail")
            # Plot baseline area
            plt.fill_between(baseline_s,
                             0,
                             maxdur, 
                             facecolor="green", 
                             color="green",
                             alpha=0.3,
                             label="Baseline")

            # Plot degradation areas
            for degradation in rel_degradation:
                plt.fill_between(degradation,
                                 0,
                                 maxdur, 
                                 facecolor="yellow", 
                                 color="yellow", 
                                 alpha=0.3)

            # Plot Degradation Threshold
            d_treshold, = plt.plot(all_seconds,
                                   mean_dur,
                                   '-', 
                                   linewidth=1.5, 
                                   color="cyan", 
                                   label="Degradation Threshold")

            # Plot Mean Duration
            mean_dur_line, = plt.plot(all_seconds, 
                                      mv_avg,
                                      '-',
                                      linewidth=1.5, 
                                      color="orange", 
                                      label="Mean Duration")

            plt.fill_between([failure_injected - base_ts[0]],
                             0,
                             maxdur, 
                             facecolor="purple",
                             color="purple")
            
            # Axes limits and labels
            plt.xlim(0, base_ts[-1]-base_ts[0] + 3)
            plt.ylim(min(all_dur), maxdur)
            plt.xlabel("Time, s")
            plt.ylabel("Operation Duration, s")

            # Areas legends
            yellow_patch = mpatches.Patch(color='Yellow', 
                                          alpha=0.3,
                                          label='Degradation')
            red_patch = mpatches.Patch(color='red',
                                       alpha=0.3, 
                                       label='Downtime')
            green_patch = mpatches.Patch(color='green', 
                                         alpha=0.3,
                                         label='Baseline')
            purple_patch = mpatches.Patch(color='purple', 
                                          label='Failure injection')

            # Legend Box
            plt.legend(bbox_to_anchor=(1.05, 1),
                       loc=2, borderaxespad=0.,
                       handles=[success_ops,
                                fail_ops,
                                d_treshold,
                                mean_dur_line,
                                yellow_patch,
                                red_patch,
                                green_patch,
                                purple_patch])

            # Set the title
            plt.title("{}\n{}({}):{}".format(task_id,
                                             scenario,
                                             str(data_index+1),
                                             failure_description))

            # show the chart
            plt.show()
        
        except:
            print ("Failed to plot task:{} something must be wrong"
                   " with the task results".format(task_id))
            continue